In [1]:
%matplotlib inline
from os import listdir, path
from requests import get
import urllib3
from lxml import etree
import pandas as pd
import warnings
from tqdm import tqdm
import itertools
warnings.filterwarnings('ignore')
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
for year in tqdm(range(2011, 2018)):
    stop = False
    assemblyNr = 0
    while stop == False:
        assemblyNr += 1
        for debateNr in range(1, 200):
            file = "h-tk-%s-%s-%s" %(str(year)+str(year+1), str(assemblyNr), str(debateNr))
            if not path.isfile('HandelingenTK/%s.xml' %file):
                url = "https://zoek.officielebekendmakingen.nl/%s" %file
                resp = get(url + ".xml", verify=False, allow_redirects=False)
                if resp.status_code == 200:
                    hand_file = open('HandelingenTK/%s.xml' %file, 'wb')
                    hand_file.write(resp.content)
                    hand_file.close()
                    resp = get(url+"/metadata.xml", verify=False, \
                                                        allow_redirects=False)
                    meta_file = open('HandelingenTKmeta/%smetadata.xml' %file, 'wb')
                    meta_file.write(resp.content)
                    meta_file.close()    
                elif debateNr == 1:
                    stop = True
                    break
                else:
                    break

  0%|                                                                                            | 0/7 [00:00<?, ?it/s]


NameError: name 'assembly' is not defined

In [ ]:
enddict = {}
i = 0
for file in listdir('HandelingenTK/'):
    context = etree.iterparse('HandelingenTK/' + file,  tag='spreekbeurt')
    mainSpeaker = None
    for _, elem in context:
        if elem.get('nieuw') == 'ja':
            speechCategory = "Main Speech"
            mainSpeaker = elem.findtext('spreker/naam/achternaam')
        elif elem.findtext('spreker/naam/achternaam') == mainSpeaker:
            speechCategory = "Response"
        else:
            speechCategory = "Interruption"
        surname = elem.findtext('spreker/naam/achternaam')
        party = elem.findtext('spreker/politiek')
        text = ' '.join(elem.find('tekst').itertext())
        enddict[i] = [surname, party, text, file, speechCategory]
        del surname, party, text, speechCategory
        i += 1
df = pd.DataFrame.from_dict(enddict, orient='index')
df = df.rename(columns={0:'surname', 1:'party', 2:'text', 3:'file', 4:'speech category'})

In [ ]:
enddict = {}
for file in listdir('HandelingenTKmeta'):
    tags = []
    datum = None
    context = etree.iterparse('HandelingenTKmeta/' + file,  tag='metadata')
    for _, elem in context:
        if elem.get('name') == "OVERHEIDop.datumVergadering":
            datum = elem.get('content')
        elif elem.get('name') == "OVERHEID.category":
            tags.append(elem.get('content'))
        enddict[file[:-12]+'.xml'] = [datum, tags]
df2 = pd.DataFrame.from_dict(enddict, orient='index')
df2 = df2.rename(columns={0:'date', 1:'tags'})

In [ ]:
combineddf = df.join(df2, on='file')
combineddf = combineddf.dropna()
#https://stackoverflow.com/questions/29370057/select-dataframe-rows-between-two-dates
combineddf['date'] = pd.to_datetime(combineddf['date'])
mask = (combineddf['date'] > '2012-11-05') & (combineddf['date'] <= '2017-03-23')
combineddf = combineddf.loc[mask]
combineddf = combineddf[combineddf['party'].isin(['50PLUS', 'CDA','ChristenUnie','D66','GroenLinks','PVV','PvdA','PvdD','SGP','SP','VVD'])]
combineddf.to_csv("Handelingen.csv")
combineddf.head(5)